# Arquitectura, modelado y gestión de datos en Data Science
<hr />

## Semana 2 - Ejercicio práctico 2 - Grupo 10
### Realizado por:
💻 Mayra Cecilia Salazar Grandes   
💻 José Manuel Espinoza Bone

# 0️⃣ Inicializar ambiente

Aquí instalamos las dependencias externas que no se encuentran en la biblioteca estándar de Python para lograr que el presente notebook se puede ejecutar sin problemas.

In [ ]:
import sys
import subprocess
# Función que permite ejecutar un comando del sistema. Recibe una list de string con las partes del comando.
# ejemplo: ["ls", "-l"] es equivalente a ejecutar "ls -l".
def RunCommand(commandList: list[str]):
    print("    ⏳ Ejecutando: ", " ".join(commandList))
    result = subprocess.run(commandList, stdout=subprocess.DEVNULL,stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        print(result.stderr) # Si existió error, mostrar en la salida estándar.

print("🟦 Instalando las dependencias externas")
RunCommand([sys.executable, "-m", "pip", "install", "numpy"]) # python -m pip install numpy
RunCommand([sys.executable, "-m", "pip", "install", "pandas"])
RunCommand([sys.executable, "-m", "pip", "install", "tabulate"])
RunCommand([sys.executable, "-m", "pip", "install", "matplotlib"])
RunCommand([sys.executable, "-m", "pip", "install", "seaborn"])
RunCommand([sys.executable, "-m", "pip", "install", "requests"])
RunCommand([sys.executable, "-m", "pip", "install", "openpyxl"]) # Necesaria para leer y escribir los archivos de Excel. Dependencia necesaria en pandas.

#Importando las dependencias
import pandas
import numpy 
import requests
import matplotlib.pyplot as pyplot
import seaborn
from tabulate import tabulate

🟦 Instalando las dependencias externas
    ⏳ Ejecutando:  c:\Users\Megam\AppData\Local\Programs\Python\Python313\python.exe -m pip install numpy
    ⏳ Ejecutando:  c:\Users\Megam\AppData\Local\Programs\Python\Python313\python.exe -m pip install pandas
    ⏳ Ejecutando:  c:\Users\Megam\AppData\Local\Programs\Python\Python313\python.exe -m pip install tabulate
    ⏳ Ejecutando:  c:\Users\Megam\AppData\Local\Programs\Python\Python313\python.exe -m pip install matplotlib
    ⏳ Ejecutando:  c:\Users\Megam\AppData\Local\Programs\Python\Python313\python.exe -m pip install seaborn
    ⏳ Ejecutando:  c:\Users\Megam\AppData\Local\Programs\Python\Python313\python.exe -m pip install requests
    ⏳ Ejecutando:  c:\Users\Megam\AppData\Local\Programs\Python\Python313\python.exe -m pip install openpyxl


# 1️⃣ Selección de la fuente de datos

Dada la problemática actual de inseguridad y violencia en nuestro país Ecuador se ha seleccionado como fuente de datos la información que publica el Ministerio del Interior y Policía Nacional en el contexto de Homicidios intencionales. Se escogió esta fuente ya que hay mucha desinformación sobre las muertes intencionales de parte de personas en redes sociales, medios de comunicación y entes políticos. Se habla de mucha inseguridad y violencia por una parte; y por otra se habla que todo está mejorando y que actualmente las muertes han disminuido significativamente. 

El ministerio comparte a julio de 2025 varias bases de datos que se actualizan en tiempo real, estas bases de datos se descargan en un archivo comprimido `HOMICIDIOS_INTENCIONALES_JUNIO_2025.rar` que contiene 2 archivos de Excel.
Se las puede descargar del siguiente sitio, haciendo click en el botón de descarga de base de datos: https://cifras.ministeriodelinterior.gob.ec/comisioncifras/#/app/estadisticas-seguridad-homicidios   
- Tenemos una base de datos en Excel de las muertes intencionales del primer semestre del año actual - ```mdi_homicidios_intencionales_pm_2025_ene_jun.xlsx```.
- Tenemos otra base de datos en Excel de las muertes intencionales desde el año 2014 hasta el año anterior al actual - ```mdi_homicidios_intencionales_pm_2014_2024.xlsx```.

Este conjunto de datos también se puede encontrar en la siguiente URI: https://datosabiertos.gob.ec/dataset/homicidios-intencionales

Se espera investigar las provincias con más muertes, los cantones con más muertes, el tipo de arma más utilizada para matar entre otros insights que se puede uno inspirar y obtener.

# 2️⃣ Adquisión de datos

In [ ]:
# Descarga de archivos, lectura y combinación con pandas
homicidios_2025_uri = "https://datosabiertos.gob.ec/dataset/0ec65ab4-e6ab-40ab-aab9-c91e912f9faf/resource/cb8f704e-2b27-4d7f-9431-d40c4e27fa48/download/mdi_homicidios_intencionales_pm_2025_ene_jun.xlsx"
homicidios_2014_2024_uri = "https://datosabiertos.gob.ec/dataset/0ec65ab4-e6ab-40ab-aab9-c91e912f9faf/resource/36b055c8-e10c-4e57-ba25-3046ca5ef15d/download/mdi_homicidios_intencionales_pm_2014_2024.xlsx"
print("🟦 Descargando orígenes de datos")

# En los archivos de Excel existe una fila en blanco al inicio, por eso saltamos una fila con skiprows=1
# Con sheet_name=1 tomamos la hoja de índice 1 en el archivo de Excel, ya que la primera hoja es de información.
print(f"    🌎 Descargando: {homicidios_2025_uri}")
df_2025 = pandas.read_excel(homicidios_2025_uri, sheet_name=1, skiprows=1)
print(f"    🌎 Descargando: {homicidios_2014_2024_uri}")
df_2014_2024 = pandas.read_excel(homicidios_2014_2024_uri, sheet_name=1, skiprows=1)

# El DataFrame de 2025 tiene 32 columnas, falta la columna si es que la persona asesinada registra antecedentes penales.
# Adicional tiene una columna en blanco al inicio la cual se va a ignorar/eliminar en la asignación
# Estamos tomando todas las filas; y las columnas desde el índice uno hasta la columna final.
df_2025 = df_2025.iloc[:, 1:]

# El DataFrame de 2014-2024 tiene 33 columnas, tiene la columna si es que registra antecedentes penales.
# Adicional tiene una columna en blanco al inicio la cual se va a ignorar/eliminar en la asignación
# Estamos tomando todas las filas; y las columnas desde el índice uno hasta la penúltima columna.
# Se hace aquí esta transformación por única vez, para evitar hacer copias destructivas sobre la misma variable ya que al ejecutar esta línea a cada momento se va eliminando una columna al final y por este motivo es la excepción no colocarla en la sección 3.
df_2014_2024 = df_2014_2024.iloc[:, 1:-1]

# Igualadas las estructuras procedemos a realizar la concatenación en vertical. A partir de este momento este es nuestro DataFrame a trabajar.
dfMain = pandas.concat([df_2025, df_2014_2024])
print("    ✅ Datos cargados en memoria")

🟦 Descargando orígenes de datos
    🌎 Descargando: https://datosabiertos.gob.ec/dataset/0ec65ab4-e6ab-40ab-aab9-c91e912f9faf/resource/cb8f704e-2b27-4d7f-9431-d40c4e27fa48/download/mdi_homicidios_intencionales_pm_2025_ene_jun.xlsx
    🌎 Descargando: https://datosabiertos.gob.ec/dataset/0ec65ab4-e6ab-40ab-aab9-c91e912f9faf/resource/36b055c8-e10c-4e57-ba25-3046ca5ef15d/download/mdi_homicidios_intencionales_pm_2014_2024.xlsx
    ✅ Datos cargados en memoria


# 3️⃣ Limpieza y exploración inicial (Análisis exploratorio de datos - EDA)

In [ ]:
## Función para mostrar la información del DataFrame
def ShowTableInfo(df:pandas.DataFrame, title):
    display(f"ℹ️ Información de DataFrame: {title} ℹ️")
    df.info()
    display()

# Función para mostrar las n primeras filas del DataFrame.
def ShowTableHead(df:pandas.DataFrame, headQty=10):
    display(f"ℹ️ Primeros {headQty} elementos.")
    print(tabulate(dfMain.head(headQty), headers="keys", tablefmt="fancy_grid"))
    display()


# Función para mostrar las n últimas filas del DataFrame.
def ShowTableTail(df:pandas.DataFrame, tailQty=10):
    display(f"ℹ️ Últimos {tailQty} elementos.")
    print(tabulate(dfMain.tail(tailQty), headers="keys", tablefmt="fancy_grid"))
    display()

# Función para mostrar la estadística descriptiva de todas las columnas del DataFrame, por tipo de dato.
def ShowTableStats(df:pandas.DataFrame):
    display(f"ℹ️ Estadística descriptiva de DataFrame")
    print("    🔢 Columnas numéricas")
    numeric_desc = df.describe(include=[numpy.number]).round(2)
    print(tabulate(numeric_desc, headers='keys', tablefmt='fancy_grid'))
    print("    🔡 Columnas no numéricas")
    non_numeric_desc = df.describe(include=['object', 'bool', 'category'])
    print(tabulate(non_numeric_desc, headers='keys', tablefmt='fancy_grid'))
    print("    📅 Columnas fechas")
    datetime_desc = df.describe(include=['datetime'])
    if not datetime_desc.empty:
        print(tabulate(datetime_desc, headers='keys', tablefmt='fancy_grid'))
    display()

# Función para mostrar los valores nulos o NaN de cada columna en un DataFrame
def ShowNanValues(df: pandas.DataFrame):
    display(f"ℹ️ Contador de valores Nulos/NaN")
    display(df.isnull().sum())
    display()
    

# EXPLORACIÓN INICIAL
ShowTableInfo(dfMain, "Homicidios 2014 - 2025, Análisis preliminar")
ShowTableStats(dfMain)
ShowTableHead(dfMain)
ShowNanValues(dfMain)

# LIMPIEZA Y CORRECCIONES
# La columna "codigo subcircuito" tiene un nombre con espacios por lo que se va a corregir su nombre. Inplace en True para evitar hacer copias innecesarias y afectar directamente el DataFrame.
dfMain.rename(columns={"codigo subcircuito": "codigo_subcircuito"}, inplace=True)

# Conversión de tipos de datos
dfMain["tipo_muerte"]=dfMain["tipo_muerte"]
dfMain["zona"]=
dfMain["subzona"]=
dfMain["distrito"]=
dfMain["circuito"]=
dfMain["codigo subcircuito"]=
dfMain["subcircuito"]=
dfMain["codigo_provincia"]=
dfMain["provincia"]=
dfMain["codigo_canton"]=
dfMain["canton"]=
dfMain["coordenada_y"]=
dfMain["coordenada_x"]=
dfMain["area_hecho"]=
dfMain["lugar"]=
dfMain["tipo_lugar"]=
dfMain["fecha_infraccion"]=
dfMain["hora_infraccion"]=
dfMain["arma"]=
dfMain["tipo_arma"]=
dfMain["presunta_motivacion"]=
dfMain["presun_motiva_observada"]=
dfMain["probable_causa_motivada"]=
dfMain["edad"]=
dfMain["medida_edad"]=
dfMain["sexo"]=
dfMain["genero"]=
dfMain["etnia"]=
dfMain["estado_civil"]=
dfMain["nacionalidad"]=
dfMain["discapacidad"]=
dfMain["profesion_registro_civil"]=
dfMain["instruccion"]=





'ℹ️ Información de DataFrame: Homicidios 2014 - 2025, Análisis preliminar ℹ️'

<class 'pandas.core.frame.DataFrame'>
Index: 35143 entries, 0 to 30523
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   tipo_muerte               35143 non-null  object        
 1   zona                      35143 non-null  object        
 2   subzona                   35143 non-null  object        
 3   distrito                  35143 non-null  object        
 4   circuito                  35143 non-null  object        
 5   codigo_subcircuito        35143 non-null  object        
 6   subcircuito               35143 non-null  object        
 7   codigo_provincia          35143 non-null  int64         
 8   provincia                 35143 non-null  object        
 9   codigo_canton             35143 non-null  int64         
 10  canton                    35143 non-null  object        
 11  coordenada_y              35143 non-null  object        
 12  coordenada_x           

'ℹ️ Estadística descriptiva de DataFrame'

    🔢 Columnas numéricas
╒═══════╤════════════════════╤═════════════════╕
│       │   codigo_provincia │   codigo_canton │
╞═══════╪════════════════════╪═════════════════╡
│ count │           35143    │        35143    │
├───────┼────────────────────┼─────────────────┤
│ mean  │              10.93 │         1096.96 │
├───────┼────────────────────┼─────────────────┤
│ std   │               4.54 │          453.04 │
├───────┼────────────────────┼─────────────────┤
│ min   │               1    │          101    │
├───────┼────────────────────┼─────────────────┤
│ 25%   │               9    │          901    │
├───────┼────────────────────┼─────────────────┤
│ 50%   │               9    │          907    │
├───────┼────────────────────┼─────────────────┤
│ 75%   │              13    │         1301    │
├───────┼────────────────────┼─────────────────┤
│ max   │              24    │         2403    │
╘═══════╧════════════════════╧═════════════════╛
    🔡 Columnas no numéricas
╒════════╤══════

'ℹ️ Primeros 10 elementos.'

╒════╤═══════════════╤════════╤════════════════╤═══════════════════╤══════════════════╤══════════════════════╤════════════════════╤════════════════════╤═════════════╤═════════════════╤════════════╤════════════════╤════════════════╤══════════════╤═══════════════╤══════════════╤═════════════════════╤═══════════════════╤══════════════════╤══════════════════╤═══════════════════════╤═══════════════════════════════════════════╤═══════════════════════════╤══════════╤═══════════════╤════════╤═══════════╤═══════════╤════════════════╤════════════════╤════════════════╤════════════════════════════╤═══════════════╕
│    │ tipo_muerte   │ zona   │ subzona        │ distrito          │ circuito         │ codigo_subcircuito   │ subcircuito        │   codigo_provincia │ provincia   │   codigo_canton │ canton     │   coordenada_y │   coordenada_x │ area_hecho   │ lugar         │ tipo_lugar   │ fecha_infraccion    │ hora_infraccion   │ arma             │ tipo_arma        │ presunta_motivacion   │ presun_m

'ℹ️ Contador de valores Nulos/NaN'

tipo_muerte                 0
zona                        0
subzona                     0
distrito                    0
circuito                    0
codigo_subcircuito          0
subcircuito                 0
codigo_provincia            0
provincia                   0
codigo_canton               0
canton                      0
coordenada_y                0
coordenada_x                0
area_hecho                  0
lugar                       0
tipo_lugar                  0
fecha_infraccion            0
hora_infraccion             0
arma                        0
tipo_arma                   0
presunta_motivacion         0
presun_motiva_observada     0
probable_causa_motivada     0
edad                        0
medida_edad                 0
sexo                        0
genero                      0
etnia                       0
estado_civil                0
nacionalidad                0
discapacidad                0
profesion_registro_civil    0
instruccion                 0
dtype: int